# Overloading Warning System
### Rayan Aryaan



In [1]:
import sys
print(sys.version)

3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]


In [2]:
# https://cloudxlab.com/blog/object-detection-yolo-and-python-pydarknet/
import numpy as np
import time
import cv2
import imutils
from imutils.video import FPS
from imutils.video import VideoStream
import pandas as pd
import re
import beepy as beep
from threading import Thread

In [3]:
# Project constants
TOTAL_BEARING_CAPACITY = 150 # Change this value based on requirement or pass this as parameter
playing_alert = False

In [4]:
# Yolo models related pre-trained data files (Object detection including human)
LABELS_FILE='data/coco.names'
CONFIG_FILE='cfg/yolov3.cfg'
WEIGHTS_FILE='Models/yolov3.weights'
CONFIDENCE_THRESHOLD=0.3
LABELS = open(LABELS_FILE).read().strip().split("\n")

# Generate random colors for labels
np.random.seed(4)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")


In [5]:
# Age Model 
AGE_WEIGHTS = "Models/age_deploy.prototxt"
AGE_CONFIG = "Models/age_net.caffemodel"

# Model requirements for image
AGE_LIST = ['(0-2)', '(4-6)', '(8-12)', '(15-20)',
        '(25-32)', '(38-43)', '(48-53)', '(60-100)']
MODEL_MEAN = (78.4263377603, 87.7689143744, 114.895847746)


In [6]:
# The gender model architecture
# https://drive.google.com/open?id=1W_moLzMlGiELyPxWiYQJ9KFaXroQ_NFQ
GENDER_MODEL = 'Models/gender_deploy.prototxt' # deploy_gender.prototxt'
# The gender model pre-trained weights
# https://drive.google.com/open?id=1AW3WduLk1haTVAxHOkVS_BEzel1WXQHP
GENDER_PROTO = 'Models/gender_net.caffemodel'
# Each Caffe Model impose the shape of the input image also image preprocessing is required like mean
# substraction to eliminate the effect of illunination changes
# Represent the gender classes
GENDER_LIST = ['Male', 'Female']


In [7]:
# play alert sound
def play_warning():
    global playing_alert
    playing_alert = True
    for ii in range(5,7): 
        beep.beep(ii)
    playing_alert = False

def play_alert_sound():
    global playing_alert
    if not playing_alert:
        thread = Thread(target=play_warning)
        thread.start()


In [8]:
# load weigth predictor dataframe (hardcoded as of now)
df = pd.read_csv("age_gender_wt_mapper.csv")
# Let's print data after column rename
df


,age_lower_limit,age_upper_limit,gender,weight
0,0,3,Male,10
1,4,6,Male,18
2,7,12,Male,35
3,13,20,Male,58
4,21,32,Male,60
5,33,43,Male,65
6,44,53,Male,68
7,54,100,Male,60
8,0,3,Female,9
9,4,6,Female,16


In [9]:
# changing gender column value. male to 0 and female to 1
df.loc[df["gender"] == "Male", "gender"] = 0
df.loc[df["gender"] == "Female", "gender"] = 1
df

,age_lower_limit,age_upper_limit,gender,weight
0,0,3,0,10
1,4,6,0,18
2,7,12,0,35
3,13,20,0,58
4,21,32,0,60
5,33,43,0,65
6,44,53,0,68
7,54,100,0,60
8,0,3,1,9
9,4,6,1,16


In [10]:
# Split the range and get min-max
def age_minmax_range(age_range_txt):
    res = re.split('\(|-|\)', age_range_txt)
    return (int(res[1]), int(res[2]))

In [11]:
# Calculate overall weight
def calculate_load(idxs, age_gender_index_list):
    # ensure at least one detection exists
    total_weight = 0
    person_count = 0
    if len(idxs) > 0:
        # loop over the indexes we are keeping
        for i in idxs.flatten():
            if age_gender_index_list[i][1] >= 0:
                # object is an human being
                person_count = person_count + 1
            if age_gender_index_list[i][1] >= 0 and age_gender_index_list[i][0] >= 0:
                age_range = age_minmax_range(AGE_LIST[age_gender_index_list[i][0]])
                # print(age_range)
                for index in df.index:
                    if age_range[0] >= df['age_lower_limit'][index] and age_range[1] <= df['age_upper_limit'][index] and age_gender_index_list[i][1] == df['gender'][index]:
                        total_weight = total_weight + df['weight'][index]
    return person_count, total_weight

In [12]:
# Load all required models

# Load gender prediction model
age_Net = cv2.dnn.readNet(AGE_CONFIG, AGE_WEIGHTS)
# Load gender prediction model
gender_net = cv2.dnn.readNetFromCaffe(GENDER_MODEL, GENDER_PROTO)
# Load object detection prediction model
object_detection_net = cv2.dnn.readNetFromDarknet(CONFIG_FILE, WEIGHTS_FILE)
# determine only the *output* layer names that we need from YOLO
ln = object_detection_net.getLayerNames()
ln = [ln[i - 1] for i in object_detection_net.getUnconnectedOutLayers()]

In [13]:
# Open camera
vs = cv2.VideoCapture(0)
# Performance check
fps = FPS().start()

In [14]:
# predict age using age net

def predict_age(blob):
    age_Net.setInput(blob)
    age_preds = age_Net.forward()
    return age_preds[0].argmax()   

In [15]:
# Predict gender using gender net
def predict_gender(blob):
    gender_net.setInput(blob)
    gender_preds = gender_net.forward()
    return gender_preds[0].argmax()


In [16]:
# process the input video (camera feed)

def process_video_input():
    while True:
        try:
            (grabbed, image) = vs.read()
        except:
            print('Unable to read camera output. Exiting')
            break
        blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
            swapRB=True, crop=False)
        object_detection_net.setInput(blob)
        (image_height, image_width) = image.shape[:2]
        layerOutputs = object_detection_net.forward(ln)
        # initialize our lists of detected bounding boxes, confidences, and
        # class IDs, respectively
        object_boxes = []
        confidences = []
        classIDs = []
        age_gender_index_list = []
        # loop over each of the layer outputs
        for output in layerOutputs:
            # loop over each of the detections
            for detection in output:
                # extract the class ID and confidence (i.e., probability) of
                # the current object detection
                scores = detection[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]
                # filter out weak predictions by ensuring the detected
                # probability is greater than the minimum probability
                if confidence > CONFIDENCE_THRESHOLD:
                    # scale the bounding box coordinates back relative to the
                    # size of the image, keeping in mind that YOLO actually
                    # returns the center (x, y)-coordinates of the bounding
                    # box followed by the boxes' width and height
                    box = detection[0:4] * np.array([image_width, image_height, image_width, image_height])
                    (centerX, centerY, width, height) = box.astype("int")
                    # use the center (x, y)-coordinates to derive the top and
                    # and left corner of the bounding box
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))
                    age_index = -1
                    gener_index = -1
                    if classID == 0 and x > 0 and y > 0 and int(width) > 0 and int(height) > 0: # Person
                        face_box = [x, y, x+int(width), y+int(height)]
                        face = image[face_box[1]:face_box[3], face_box[0]:face_box[2]]
                        # ----- Image preprocessing --------#
                        blob = cv2.dnn.blobFromImage(
                            face, 1.0, (227, 227), MODEL_MEAN, swapRB=False)
                        # Age Prediction
                        age_index = predict_age(blob)
                        # Predict Gender
                        gener_index = predict_gender(blob)
                    # update our list of bounding box coordinates, confidences,
                    # and class IDs
                    object_boxes.append([x, y, int(width), int(height)])
                    age_gender_index_list.append([age_index, gener_index])
                    confidences.append(float(confidence))
                    classIDs.append(classID)
        # apply non-maxima suppression to suppress weak, overlapping bounding boxes
        idxs = cv2.dnn.NMSBoxes(object_boxes, confidences, CONFIDENCE_THRESHOLD, CONFIDENCE_THRESHOLD)

        # ensure at least one detection exists
        if len(idxs) > 0:
            # loop over the indexes we are keeping
            for i in idxs.flatten():
                # extract the bounding box coordinates
                (x, y) = (object_boxes[i][0], object_boxes[i][1])
                (w, h) = (object_boxes[i][2], object_boxes[i][3])

                color = [int(c) for c in COLORS[classIDs[i]]]

                cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
                text = "{}".format(LABELS[classIDs[i]])
#                 if age_gender_index_list[i][1] >= 0:
#                     text = "{}, {}".format(text, GENDER_LIST[age_gender_index_list[i][1]])
#                 if age_gender_index_list[i][0] >= 0:
#                     text = "{}, Age - {}".format(text, AGE_LIST[age_gender_index_list[i][0]])
                cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, color, 2)
        text = "No of objects: {}".format(len(idxs))
        no_person, total_weight = calculate_load(idxs, age_gender_index_list)
        if total_weight > TOTAL_BEARING_CAPACITY:
            # send alert message, audio/visual message.
            print(f'Exceed capacity. No of person - {no_person}. Bearing capacity - {TOTAL_BEARING_CAPACITY}. Total estimated wt - {total_weight}')
            play_alert_sound()
        cv2.putText(image, text, (15, 15), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (255, 255, 255), 2)
        # show the output image
        cv2.imshow("output", cv2.resize(image,(800, 600)))
        fps.update()
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break



In [17]:
# Release all the occupied resources
def release_video_input():
    fps.stop()
    print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
    # do a bit of cleanup
    cv2.destroyAllWindows()
    # release camera
    vs.release()

In [ ]:
# let's execute
if __name__ == '__main__':
    process_video_input()
    release_video_input()

Exceed capacity. No of person - 4. Bearing capacity - 150. Total estimated wt - 207
Exceed capacity. No of person - 5. Bearing capacity - 150. Total estimated wt - 217
Exceed capacity. No of person - 3. Bearing capacity - 150. Total estimated wt - 163
Exceed capacity. No of person - 4. Bearing capacity - 150. Total estimated wt - 193
Exceed capacity. No of person - 3. Bearing capacity - 150. Total estimated wt - 161
Exceed capacity. No of person - 3. Bearing capacity - 150. Total estimated wt - 175
Exceed capacity. No of person - 4. Bearing capacity - 150. Total estimated wt - 234
Exceed capacity. No of person - 3. Bearing capacity - 150. Total estimated wt - 173
Exceed capacity. No of person - 4. Bearing capacity - 150. Total estimated wt - 172
Exceed capacity. No of person - 5. Bearing capacity - 150. Total estimated wt - 242
Exceed capacity. No of person - 7. Bearing capacity - 150. Total estimated wt - 354
Exceed capacity. No of person - 6. Bearing capacity - 150. Total estimated w